In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os
import pickle
import socket
import struct
import time
import cv2
from socket import SOCK_SEQPACKET
from queue import Queue
import threading
import sqlite3 as db
import datetime
maskDB = db.connect("maskDB.db")
cur = maskDB.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS maskData(srno INTEGER PRIMARY KEY AUTOINCREMENT, date DATE, time TIME, address TEXT, temp INTEGER, imageLoc TEXT)")
maskDB.commit()

In [2]:
def checkMask(frame):
    
    prototxtPath = r"C:\Users\vasu2\Desktop\Mask_Detection_BE\face_detector\deploy.prototxt"
    weightsPath = r"C:\Users\vasu2\Desktop\Mask_Detection_BE\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
    faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

    maskNet = load_model(r"C:\Users\vasu2\Desktop\Mask Detection Test\maskdata\MaskDetectionModel")


    frame = imutils.resize(frame, width=400)
    assert not isinstance(frame,type(None)), 'frame not found'
    preds = detect_and_predict_mask(frame, faceNet, maskNet)

    
    pred = zip(preds)

    try:
        str = list(pred)[0][0]
    except:
        return 3
    
    if str[0] > str[1]:
        return 1
    else:
        return 2

In [3]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
        (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    faces = []
    locs = []
    preds = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
        
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    
            
            
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))
            
        
            
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
    return preds

In [4]:
HOST = '192.168.0.126'
PORT = 8089
global conn

a = "MASK!"
b = "NO MASK!"
c = "FACE NOT DETECTED!"
result = ""


def createSocket():
    global s
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        print('Socket created')
    except socket.error as e:
        print("Error: " + str(e))

        
        
def socketBind():
    global s
    try:
        s.bind((HOST, PORT))
        print('Socket bind complete')
    except socket.error as e:
        print("Error: " + str(e))


        
        
def socketListen():
    global s
    global conn
    global address
    
    s.listen(10)
    conn, address = s.accept()
    print(f"Connection from {address} has been established.")




def reteriveData():
    global s
    global conn
    global address
    global data
    global frame_data
    rawCode = ''
    code = ''
    rawTemp = ''
    temp = ''
    
    rawCode = conn.recv(8)
    code = rawCode.decode()
        
        
    data = b'' ### CHANGED
    payload_size = struct.calcsize("L") ### CHANGED
    # Retrieve message size
    while len(data) < payload_size:
        data += conn.recv(4096)

    packed_msg_size = data[:payload_size]
    data = data[payload_size:]
    msg_size = struct.unpack("L", packed_msg_size)[0] ### CHANGED

    # Retrieve all data based on message size
    while len(data) < msg_size:
        data += conn.recv(4096)

    frame_data = data[:msg_size]
    data = data[msg_size:]
    
    
    print(code)   
    if code == 'x':
        extractFrame()
        sendReply()
        
    else:
        #rawTemp = conn.recv(8)
      
        #temp = rawTemp.decode()
        conn.close()
        storeData(frame_data, code)
        
def storeData(frame_data, temp):
    global address
    frame = pickle.loads(frame_data)
    print(temp)
    path = r"C:\Users\vasu2\Desktop\noMaskImages"
    dateTimeList = []
    dateTimeList = str(datetime.datetime.now()).split(" ")
    loc = dateTimeList[0] +"-"+ dateTimeList[1].split(".")[0].replace(":","-") + ".bmp"
    print(cv2.imwrite(str(os.path.join(path , loc)),frame))
    cur.execute("INSERT INTO maskData(date,time,address,temp,imageLoc) VALUES (?,?,?,?,?);", (dateTimeList[0],dateTimeList[1],str(address),str(temp),loc))
    maskDB.commit()
    showDataBase()
        
def showDataBase():
    cur.execute("SELECT * FROM maskData")
    while True:
        record=cur.fetchone()
        if record==None:
            break
        print (record)

    
def extractFrame():
    global s
    global frame_data
    global result
    # Extract frame
    t0= time.time()
    frame = pickle.loads(frame_data)
    
    t1= time.time()
    i = checkMask(frame)
    print("\nPickle Time: ", t1-t0)
    print("\nCheck Time: ", time.time()-t1,"\n\n")
    
    print (i)
    if i == 1:
        result = a
    elif i == 2:
        result = b
    elif i == 3:
        result = c
    
    

def sendReply():
    global conn
    global address
    global result
    print (result)
    conn.send(bytes(result,"utf-8"))
    conn.close()

        
def executeSystem():
    createSocket()
    socketBind()
    
    
    while True:
        socketListen()
        reteriveData()

    #reateThreads()
    #reateJobs()

In [ ]:
executeSystem()

Socket created
Socket bind complete
Connection from ('192.168.0.163', 34892) has been established.
x

Pickle Time:  0.0

Check Time:  1.6151454448699951 


2
NO MASK!
Connection from ('192.168.0.163', 34894) has been established.
84
84
True
(1, '2021-03-28', '23:24:39.685779', "('192.168.0.163', 59874)", 71, 'jjf')
(2, '2021-03-28', '23:25:47.981854', "('192.168.0.163', 60004)", 70, 'jjf')
(3, '2021-03-28', '23:44:24.730362', "('192.168.0.163', 60660)", 77, '2021-03-28 23:44:24.730362.jpg')
(4, '2021-03-28', '23:49:43.209210', "('192.168.0.163', 60666)", 79, '2021-03-28 23:49:43.209210.jpg')
(5, '2021-03-28', '23:53:47.912482', "('192.168.0.163', 32810)", 80, '2021-03-28 23:53:47.913453.jpg')
(6, '2021-03-28', '23:54:12.438094', "('192.168.0.163', 32940)", 80, '2021-03-28 23:54:12.438094.jpg')
(7, '2021-03-28', '23:54:17.709552', "('192.168.0.163', 32944)", 80, '2021-03-28 23:54:17.709552.jpg')
(8, '2021-03-28', '23:54:24.951381', "('192.168.0.163', 32948)", 80, '2021-03-28 23:54:24.95

In [ ]:
# socket -> bind -> listen -> accept


# receiveframe - > checkformask -> reply


# rpi -> connect -> framesend -> reply -> condition



# 1) time -> 30 - 40sec and sensor issue
# 2) time -> 4sec ( system was designed to check for mask twice)

In [ ]:
dateTimeList = []
dateTimeList = str(datetime.datetime.now()).split(" ")
loc = dateTimeList[0] +"-"+ dateTimeList[1].split(".")[0].replace(":","-") + ".bmp"
loc

In [ ]:
cur.ex

In [ ]:
cur.execute("DROP TABLE maskData")
maskDB.commit()

In [1]:
import sqlite3 as db
import datetime
maskDB = db.connect("maskDB.db")
cur = maskDB.cursor()

In [2]:
cur.execute("SELECT * FROM maskData")

In [3]:
while True:
        record=cur.fetchone()
        if record==None:
            break
        print (record)

(1, '2021-03-28', '23:24:39.685779', "('192.168.0.163', 59874)", 71, 'jjf')
(2, '2021-03-28', '23:25:47.981854', "('192.168.0.163', 60004)", 70, 'jjf')
(3, '2021-03-28', '23:44:24.730362', "('192.168.0.163', 60660)", 77, '2021-03-28 23:44:24.730362.jpg')
(4, '2021-03-28', '23:49:43.209210', "('192.168.0.163', 60666)", 79, '2021-03-28 23:49:43.209210.jpg')
(5, '2021-03-28', '23:53:47.912482', "('192.168.0.163', 32810)", 80, '2021-03-28 23:53:47.913453.jpg')
(6, '2021-03-28', '23:54:12.438094', "('192.168.0.163', 32940)", 80, '2021-03-28 23:54:12.438094.jpg')
(7, '2021-03-28', '23:54:17.709552', "('192.168.0.163', 32944)", 80, '2021-03-28 23:54:17.709552.jpg')
(8, '2021-03-28', '23:54:24.951381', "('192.168.0.163', 32948)", 80, '2021-03-28 23:54:24.952377.jpg')
(9, '2021-03-28', '23:54:29.303197', "('192.168.0.163', 32952)", 80, '2021-03-28 23:54:29.303197.jpg')
(10, '2021-03-28', '23:55:46.499871', "('192.168.0.163', 33170)", 80, '2021-03-28 23:55:46.499871.jpg')
(11, '2021-03-28', '23:

In [7]:
cur.execute("SELECT * FROM maskData")
rec = cur.fetchone()

In [12]:
rec

(3,
 '2021-03-28',
 '23:44:24.730362',
 "('192.168.0.163', 60660)",
 77,
 '2021-03-28 23:44:24.730362.jpg')

In [15]:
rec = cur.fetchone()
rec

(6,
 '2021-03-28',
 '23:54:12.438094',
 "('192.168.0.163', 32940)",
 80,
 '2021-03-28 23:54:12.438094.jpg')

In [16]:
rec.dtype()

AttributeError: 'tuple' object has no attribute 'dtype'